###  Model Specification
Model: pretrained resnet34 | Data: Full classes | Dropout: 0.08 | Max Size: 299 | Remarks: 3 extra fully connected layers [512, 512, 256]

Result: 



In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
sys.path.append('/home/r8user2/fastai/old')
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *
import matplotlib.pyplot as plt

In [2]:
# full datasets
PATH = '/home/r8user2/Documents/HY/dress_data/datasets'
CPU_WORKERS = 60
sz = 224

In [3]:
arch = resnet34
tfms = tfms_from_model(resnet34,
                       sz,
                       aug_tfms = transforms_side_on,
                       max_zoom = 1.1)

In [4]:
# For upgrade image size later
def get_data(sz, bs = 64):
    tfms = tfms_from_model(arch, sz, aug_tfms = transforms_side_on, max_zoom = 1.1)
    data = ImageClassifierData.from_paths(PATH,
                                          tfms = tfms,
                                          trn_name = 'imgtrain',
                                          val_name = 'imgval',
                                          num_workers = CPU_WORKERS,
                                          bs = bs)
    return data if sz>300 else data.resize(340, 'tmp')

In [5]:
data = get_data(sz)

### Find Optimal Dropout Rate

In [ ]:
# optimal dropout rate
# default ps = [0.25, 0.5]
import numpy as np
lr=1e-2

dropout_rate=[]
min_val_loss=[]
for dropout in np.arange(0.00, 0.20, 0.02):
    dropout_rate.append(dropout)
    print('Dropout rate = %s' % dropout)
    learn = ConvLearner.pretrained(arch, data, ps = dropout, precompute = True)
    _, ep_vals = learn.fit(lr, 20, get_ep_vals=True);
    min_val_loss.append(min(l[1] for l in list(ep_vals.values())))

In [ ]:
plt.plot(dropout_rate, min_val_loss)

In [6]:
learn = ConvLearner.pretrained(arch, 
                               data, 
                               ps = 0.08, 
                               xtra_fc = [512, 512, 128], 
                               precompute = True)

### Find Optimal Learning Rate

In [ ]:
# 1e-2 to 1e-3 seems to be suitable
# pick a larger one (1e-2)
learn.lr_find(1e-4, 1e0)
learn.sched.plot()

### Train last layer with precomputing

In [7]:
learn.precompute = True
%time learn.fit(1e-2, 6)

epoch      trn_loss   val_loss   accuracy                     
    0      4.257208   4.192883   0.113304  
    1      4.164121   4.0428     0.136284                     
    2      4.078451   3.945859   0.152953                     
    3      3.992427   3.879412   0.162029                     
    4      3.920734   3.822564   0.17297                      
    5      3.842676   3.775227   0.182395                     

CPU times: user 18min 5s, sys: 11min 41s, total: 29min 47s
Wall time: 18min 50s


[array([3.77523]), 0.18239496428048577]

In [9]:
%time learn.fit(1e-2, 4)

epoch      trn_loss   val_loss   accuracy                     
    0      3.80197    3.742476   0.188723  
    1      3.766348   3.71262    0.194202                     
    2      3.735387   3.685162   0.198515                     
    3      3.680031   3.662806   0.205342                     

CPU times: user 11min 55s, sys: 7min 34s, total: 19min 29s
Wall time: 12min 13s


[array([3.66281]), 0.20534212585993408]

In [10]:
learn.save('temp_resnet34_fullclass')

### Train last layer with data augmentation

In [ ]:
learn.precompute = False
%time learn.fit(1e-2, 1, cycle_len = 3)

 16%|█▋        | 1216/7433 [08:51<2:34:22,  1.49s/it, loss=3.85]

In [ ]:
learn.save('temp_resnet34_fullclass')

### Unfreeze and train all layers

In [ ]:
import numpy as np
lrs = np.array([1e-4, 1e-3, 1e-2])

In [ ]:
learn.unfreeze()
learn.precompute = False
%time learn.fit(lrs, 1, cycle_len = 4)

### Train with Bigger Sizes

In [ ]:
# sz = 299
learn.set_data(get_data(299))
learn.freeze()
learn.precompute = True
learn.save('temp_resnet34_299')

In [ ]:
learn.lr_find()

In [ ]:
learn.sched.plot()

In [ ]:
# train last layer with precompute
learn.load('temp_resnet34_299')
%time learn.fit(5e-3, 3, cycle_len = 1)

In [ ]:
%time learn.fit(5e-3, 2, cycle_len = 1, cycle_mult = 2)

In [ ]:
%time learn.fit(5e-3, 2, cycle_len = 3)

In [ ]:
learn.precompute = True
learn.freeze()
learn.load('temp_resnet34_299')

### TTA and Confusion Matrix